In [1]:
import datetime
import numpy as np
import os
import pandas as pd
import time

from copy import deepcopy
from scipy.stats import rankdata
from sklearn.metrics import log_loss
from sklearn.metrics import roc_auc_score

def timedelta(seconds):
    return str(datetime.timedelta(seconds=seconds)).split('.')[0]

os.listdir('results_DEV')

['DEV_PREDS_2020-03-03_07-25-04_efficientnet-b4-radam-lookahead-baseline_994682_ep4.csv',
 'DEV_PREDS_2020-03-12_15-55-44_efficientnet-b4-radam-lookahead-baseline-undersampling_995658_ep4.csv',
 'DEV_PREDS_2020-03-14_07-37-44_efficientnet-b3-radam-lookahead-baseline-undersampling_994986_ep3.csv',
 'DEV_PREDS_2020-03-16_08-08-14_se-resnext-50-radam-lookahead-baseline-undersampling-normalized_995047_ep4.csv',
 'DEV_PREDS_2020-03-12_12-20-58_efficientnet-b4-radam-lookahead-baseline-undersampling_995090_ep3.csv']

In [2]:
DEV_DIR = 'results_DEV'
TEST_DIR = 'results_TEST'

models_names = [
    '2020-03-03_07-25-04_efficientnet-b4-radam-lookahead-baseline_994682_ep4',
    '2020-03-12_12-20-58_efficientnet-b4-radam-lookahead-baseline-undersampling_995090_ep3',
    '2020-03-12_15-55-44_efficientnet-b4-radam-lookahead-baseline-undersampling_995658_ep4',
    '2020-03-14_07-37-44_efficientnet-b3-radam-lookahead-baseline-undersampling_994986_ep3',
    '2020-03-16_08-08-14_se-resnext-50-radam-lookahead-baseline-undersampling-normalized_995047_ep4'
]

NUM_MODELS = len(models_names)

dev_preds_files = ['DEV_PREDS_' + mn + '.csv' for mn in models_names]

test_preds_files = ['TEST_PREDS_' + mn + '.csv' for mn in models_names]

dev_target = pd.read_csv(os.path.join(DEV_DIR, dev_preds_files[0]))['target'].values
test_target = pd.read_csv(os.path.join(TEST_DIR, test_preds_files[0]))['target'].values

dev_model_preds = pd.DataFrame(np.array([pd.read_csv(os.path.join(DEV_DIR, dev_preds_files[i]))['prediction'].values for i in range(len(dev_preds_files))]).T, columns=models_names)
test_model_preds = pd.DataFrame(np.array([pd.read_csv(os.path.join(TEST_DIR, test_preds_files[i]))['prediction'].values for i in range(len(test_preds_files))]).T, columns=models_names)
dev_target.shape, dev_model_preds.shape, test_target.shape, test_model_preds.shape

((1140,), (1140, 5), (1143,), (1143, 5))

In [3]:
dev_model_preds.head()

,2020-03-03_07-25-04_efficientnet-b4-radam-lookahead-baseline_994682_ep4,2020-03-12_12-20-58_efficientnet-b4-radam-lookahead-baseline-undersampling_995090_ep3,2020-03-12_15-55-44_efficientnet-b4-radam-lookahead-baseline-undersampling_995658_ep4,2020-03-14_07-37-44_efficientnet-b3-radam-lookahead-baseline-undersampling_994986_ep3,2020-03-16_08-08-14_se-resnext-50-radam-lookahead-baseline-undersampling-normalized_995047_ep4
0,0.985469,0.716145,0.849930,0.728406,0.698786
1,0.130444,0.128722,0.002464,0.001936,0.115373
2,0.999233,0.952975,0.989828,0.993654,0.919238
3,0.999994,0.999995,0.999994,0.999999,1.000000
4,0.602908,0.351408,0.176879,0.147604,0.353536


In [4]:
test_model_preds.head()

,2020-03-03_07-25-04_efficientnet-b4-radam-lookahead-baseline_994682_ep4,2020-03-12_12-20-58_efficientnet-b4-radam-lookahead-baseline-undersampling_995090_ep3,2020-03-12_15-55-44_efficientnet-b4-radam-lookahead-baseline-undersampling_995658_ep4,2020-03-14_07-37-44_efficientnet-b3-radam-lookahead-baseline-undersampling_994986_ep3,2020-03-16_08-08-14_se-resnext-50-radam-lookahead-baseline-undersampling-normalized_995047_ep4
0,0.999989,0.999993,0.999996,1.000000,1.000000
1,0.998493,0.993685,0.999579,0.985782,0.996469
2,0.555657,0.315384,0.174179,0.060892,0.921640
3,0.999989,0.975139,0.999857,0.999937,1.000000
4,0.294504,0.140292,0.132307,0.072566,0.265427


In [5]:
dev_scores = {}
test_scores = {}
for m in models_names:
    dev_scores[m] = roc_auc_score(dev_target, dev_model_preds[m].values)
    test_scores[m] = roc_auc_score(test_target, test_model_preds[m].values)
print('DEV')
for k, v in sorted(dev_scores.items(), key=lambda x: x[1], reverse=True):
    print(k, v)
print('TEST')
for k, v in sorted(test_scores.items(), key=lambda x: x[1], reverse=True):
    print(k, v)

DEV
2020-03-14_07-37-44_efficientnet-b3-radam-lookahead-baseline-undersampling_994986_ep3 0.9983893101054896
2020-03-03_07-25-04_efficientnet-b4-radam-lookahead-baseline_994682_ep4 0.9980714107842047
2020-03-16_08-08-14_se-resnext-50-radam-lookahead-baseline-undersampling-normalized_995047_ep4 0.9980290242080333
2020-03-12_15-55-44_efficientnet-b4-radam-lookahead-baseline-undersampling_995658_ep4 0.9979813393098406
2020-03-12_12-20-58_efficientnet-b4-radam-lookahead-baseline-undersampling_995090_ep3 0.9979389527336693
TEST
2020-03-12_12-20-58_efficientnet-b4-radam-lookahead-baseline-undersampling_995090_ep3 0.99916560321979
2020-03-12_15-55-44_efficientnet-b4-radam-lookahead-baseline-undersampling_995658_ep4 0.9987790811818986
2020-03-03_07-25-04_efficientnet-b4-radam-lookahead-baseline_994682_ep4 0.9986441052321586
2020-03-14_07-37-44_efficientnet-b3-radam-lookahead-baseline-undersampling_994986_ep3 0.9984723176597624
2020-03-16_08-08-14_se-resnext-50-radam-lookahead-baseline-undersam

In [6]:
dev_scores = {}
test_scores = {}
for m in models_names:
    dev_scores[m] = log_loss(dev_target, dev_model_preds[m].values)
    test_scores[m] = log_loss(test_target, test_model_preds[m].values)
print('DEV')
for k, v in sorted(dev_scores.items(), key=lambda x: x[1], reverse=True):
    print(k, v)
print('TEST')
for k, v in sorted(test_scores.items(), key=lambda x: x[1], reverse=True):
    print(k, v)

DEV
2020-03-14_07-37-44_efficientnet-b3-radam-lookahead-baseline-undersampling_994986_ep3 0.05954510291565401
2020-03-12_12-20-58_efficientnet-b4-radam-lookahead-baseline-undersampling_995090_ep3 0.05472430936050911
2020-03-03_07-25-04_efficientnet-b4-radam-lookahead-baseline_994682_ep4 0.05164911964365804
2020-03-12_15-55-44_efficientnet-b4-radam-lookahead-baseline-undersampling_995658_ep4 0.050398067368545484
2020-03-16_08-08-14_se-resnext-50-radam-lookahead-baseline-undersampling-normalized_995047_ep4 0.049711863692772994
TEST
2020-03-14_07-37-44_efficientnet-b3-radam-lookahead-baseline-undersampling_994986_ep3 0.0660715595858408
2020-03-12_12-20-58_efficientnet-b4-radam-lookahead-baseline-undersampling_995090_ep3 0.054693871918473635
2020-03-16_08-08-14_se-resnext-50-radam-lookahead-baseline-undersampling-normalized_995047_ep4 0.052360825792158755
2020-03-12_15-55-44_efficientnet-b4-radam-lookahead-baseline-undersampling_995658_ep4 0.05230812505607138
2020-03-03_07-25-04_efficientn

In [7]:
def ensemble_predictions(predictions, weights, type_="linear"):
#     assert np.isclose(np.sum(weights), 1.0)
    if type_ == "linear":
        res = np.average(predictions, weights=weights, axis=1)
        return res
    elif type_ == "harmonic":
        res = np.average([1 / p for p in predictions.values], weights=weights, axis=1)
        return 1 / res
    elif type_ == "geometric":
        numerator = np.average(
            [np.log(p) for p in predictions.values], weights=weights, axis=1
        )
        res = np.exp(numerator / sum(weights))
        return res
    elif type_ == "rank":
        res = np.average([rankdata(p) for p in predictions.values.T], weights=weights, axis=0)
        return res / (len(res) + 1)

init_weights = [50 for _ in range(NUM_MODELS)]

def custom_predict(model_preds, weights=init_weights, type_='linear'):
    y_pred = ensemble_predictions(model_preds, weights=(np.array(weights) + 1e-15) / (sum(weights) + 1e-15), type_=type_)
    return y_pred

types = ['linear', 'harmonic', 'geometric']
res_dict = {'linear': 0, 'harmonic': 0, 'geometric': 0}
for type_ in types:
    score = roc_auc_score(dev_target, custom_predict(dev_model_preds, type_=type_))
    res_dict[type_] = score
for k, v in sorted(res_dict.items(), key=lambda x: x[1], reverse=True):
    print(k, v)

geometric 0.9987760876130529
linear 0.9987760876130529
harmonic 0.9987284027148602


In [8]:
def val_result(val_preds_fold, y_val, weights, type_):
    y_pred = custom_predict(val_preds_fold, weights, type_)
    return roc_auc_score(y_val, y_pred)
 
def go_down(best_weights, best_result, j, type_, val_preds_fold, y_val):
    curr_weights = deepcopy(best_weights)
    curr_weights[j] -= 1
    curr_result = val_result(val_preds_fold, y_val, curr_weights, type_)
    if curr_result > best_result:
        return curr_weights, curr_result
    else:
        return best_weights, best_result
    
def go_up(best_weights, best_result, j, type_, val_preds_fold, y_val):
    curr_weights = deepcopy(best_weights)
    curr_weights[j] += 1
    curr_result = val_result(val_preds_fold, y_val, curr_weights, type_)
    if curr_result > best_result:
        return curr_weights, curr_result
    else:
        return best_weights, best_result

    
def weights_tuning(
    val_preds,
    y_val,
    starting_points=[-10, 10, 30, 50, 70],
    max_iters=200,
    types=types
):
    best = {'weights': None, 'result': 0, 'type_': None, 'starting_point': None}
    for type_ in types:
        for w in starting_points:
            init_weights = [w for _ in range(NUM_MODELS)]
            best_weights = init_weights
            best_result = val_result(val_preds, y_val, best_weights, type_)
            for i in range(max_iters):
    #             print('Start of iteration #{}'.format(i + 1))
                n_changes = 0
                for j in range(len(best_weights)):
                    changed = True
                    changed_down = False
                    while changed:
                        curr_weights, curr_result = go_down(best_weights, best_result, j, type_, val_preds, y_val)
                        if curr_result == best_result:
                            changed = False
                        else:
    #                         print('Weights changed to: {}, result: {}'.format(curr_weights, curr_result))
                            best_weights, best_result = curr_weights, curr_result
                            n_changes += 1
                            changed_down = True
                    if not changed_down:
                        changed = True
                        while changed:
                            curr_weights, curr_result = go_up(best_weights, best_result, j, type_, val_preds, y_val)
                            if curr_result == best_result:
                                changed = False
                            else:
    #                             print('Weights changed to: {}, result: {}'.format(curr_weights, curr_result))
                                best_weights, best_result = curr_weights, curr_result
                                n_changes += 1
    #             print('Best weights: {}, result: {}'.format(best_weights, best_result))

    #             print('End of iteration #{}, number of changes: {}'.format(i + 1, n_changes))
    #             print()
                if n_changes == 0:
                    break
            if best_result > best['result']:
                best['result'] = best_result
                best['weights'] = best_weights
                best['type_'] = type_
                best['starting_point'] = w
    return best

In [13]:
tuning_results = []
start_time = time.time()
best = weights_tuning(dev_model_preds, dev_target, starting_points=list(range(-200, 200)))
print('(Best starting point: {})'.format(best['starting_point']))
print('Best weights:', best['weights'])
print('Best type:', best['type_'])
print('Validation score:', best['result'])
print('Time: {}'.format(timedelta(time.time() - start_time)))

(Best starting point: -5)
Best weights: [-5, -5, -2, -5, -6]
Best type: geometric
Validation score: 0.9989032473415669
Time: 0:01:06


In [14]:
roc_auc_score(test_target, custom_predict(test_model_preds, weights=best['weights'], type_=best['type_']))

0.9992576322764307

In [15]:
log_loss(test_target, custom_predict(test_model_preds, weights=best['weights'], type_=best['type_']))

0.050444072004269556